In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import time
from tensorboard.plugins.hparams import api as hp

2023-12-29 02:32:21.212824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 02:32:21.537509: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-29 02:32:21.545450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-29 02:32:21.545486: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

### 1. Model Architecture

In [2]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.0, 0.15, 0.3, 0.45]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'tanh']))
HP_LR = hp.HParam('learning_rate', hp.Discrete([0.0001, 0.001, 0.01, 0.1]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([0, 1, 2, 3, 4]))


In [3]:
def createModel(config):
  model = Sequential()
  model.add(Dense(config[HP_NUM_UNITS], activation=config[HP_ACTIVATION], input_shape=(7*7*1280,)))

  for i in range(config[HP_NUM_LAYERS]):
    model.add(Dense(config[HP_NUM_UNITS], activation=config[HP_ACTIVATION]))

  model.add(Dense(1, activation='sigmoid'))

  if config[HP_OPTIMIZER] == 'adam':
    optimizer = Adam(learning_rate=config[HP_LR])
  elif config[HP_OPTIMIZER] == 'sgd':
    optimizer = SGD(learning_rate=config[HP_LR])
  else:
    optimizer = RMSprop(learning_rate=config[HP_LR])

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

### 2. Training 

In [4]:
# Euclidean distance 
def linear_diff(np1, np2):
  np1 = np1.flatten()
  np2 = np2.flatten()

  return np1 - np2

# TODO - also try absolute difference, cosine similarity, etc.

In [5]:
# features = np.load("./train_X.npy")
# trainY = np.load("./train_y.npy")

# print(features.shape)
# trainX = []
# for i in range(0, len(features)):
#   trainX.append(linear_diff(features[i][0], features[i][1]))

# trainX = np.array(trainX)
# print(trainX.shape)
# print(trainY.shape)


# # Shuffle
# indices = np.arange(trainX.shape[0])
# np.random.shuffle(indices)
# trainX = trainX[indices]  
# trainY = trainY[indices]

In [6]:
# Save the eud version

# np.save("./train_X_eud.npy", trainX)
# np.save("./train_y_eud.npy", trainY)

# Load the eud version
trainX = np.load("./train_X_eud.npy")
trainY = np.load("./train_y_eud.npy")

In [7]:
def stringifyConfig(config):
  return f"numLayers-{config[HP_NUM_LAYERS]}_numUnits-{config[HP_NUM_UNITS]}_dropout-{config[HP_DROPOUT]}_optimizer-{config[HP_OPTIMIZER]}_activation-{config[HP_ACTIVATION]}_lr-{config[HP_LR]}"

In [8]:
method = "flatten&eudlidian"

for NUM_UNIT in HP_NUM_UNITS.domain.values:
  for NUM_LAYER in HP_NUM_LAYERS.domain.values:

    config = {
      HP_NUM_UNITS: NUM_UNIT,
      HP_DROPOUT: 0,
      HP_OPTIMIZER: 'adam',
      HP_ACTIVATION: 'relu',
      HP_LR: 0.001,
      HP_NUM_LAYERS: NUM_LAYER
    }

    # Callbacks
    modelID = int(time.time())
    modelName = stringifyConfig(config) + str(modelID)

    print("\nmodelID: ", modelID)

    # Callbacks
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=f"./logs/{method}/{modelName}", histogram_freq=1)

    checkpoint = ModelCheckpoint("./models/" + method + "/" + modelName + ".h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')

    hparms = hp.KerasCallback("./logs/" + method + "/" + modelName, config)

    callbacks_list = [checkpoint, early, tensorboard_callback, hparms]

    model = createModel(config)

    print("Parameters: ", model.count_params())
    print("HP_NUM_UNITS: ", NUM_UNIT)
    print("HP_NUM_LAYERS: ", NUM_LAYER)

    model.fit(trainX, trainY, batch_size=32, epochs=100, validation_split=0.2, callbacks=callbacks_list, shuffle=True, verbose=0)



modelID:  1703838767


2023-12-29 02:32:47.660208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-29 02:32:47.660291: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-29 02:32:47.660349: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (blankk): /proc/driver/nvidia/version does not exist
2023-12-29 02:32:47.661757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Parameters:  1003553
HP_NUM_UNITS:  16
HP_NUM_LAYERS:  0

Epoch 1: val_loss improved from inf to 1.73797, saving model to ./models/flatten&eudlidian/numLayers-0_numUnits-16_dropout-0_optimizer-adam_activation-relu_lr-0.0011703838767.h5

Epoch 2: val_loss improved from 1.73797 to 1.30235, saving model to ./models/flatten&eudlidian/numLayers-0_numUnits-16_dropout-0_optimizer-adam_activation-relu_lr-0.0011703838767.h5

Epoch 3: val_loss improved from 1.30235 to 0.80913, saving model to ./models/flatten&eudlidian/numLayers-0_numUnits-16_dropout-0_optimizer-adam_activation-relu_lr-0.0011703838767.h5

Epoch 4: val_loss did not improve from 0.80913

Epoch 5: val_loss improved from 0.80913 to 0.77240, saving model to ./models/flatten&eudlidian/numLayers-0_numUnits-16_dropout-0_optimizer-adam_activation-relu_lr-0.0011703838767.h5

Epoch 6: val_loss improved from 0.77240 to 0.76095, saving model to ./models/flatten&eudlidian/numLayers-0_numUnits-16_dropout-0_optimizer-adam_activation-relu_lr-0.0

KeyboardInterrupt: 

### Testing

In [20]:
# Load best model
model = keras.models.load_model("./models/" + method + "/" + modelName + ".h5")

In [21]:
textY = np.load("./test_y.npy") 
textX = []
features = np.load("./test_X.npy")
for i in range(0, len(features)):
  textX.append(linear_diff(features[i][0], features[i][1]))

textX = np.array(textX)

print(textX.shape)
print(textY.shape)

score = model.evaluate(textX, textY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


(1800, 62720)
(1800,)
Test loss: 0.3050694167613983
Test accuracy: 0.9127777814865112


In [ ]:
W